In [1]:
import numpy as np

# Load sudokus
sudokus = np.load("data/easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudokus.shape}, sudoku[0].shape: {sudokus[0].shape}, sudoku.dtype: {sudokus.dtype}")

# Load solutions for demonstration
solutions = np.load("data/medium_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudokus[0].tolist(), "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[8, 5, 2, 9, 7, 6, 2, 4, 3], [6, 7, 9, 1, 4, 3, 2, 8, 5], [0, 3, 1, 2, 5, 8, 7, 6, 9], [3, 1, 4, 5, 2, 7, 8, 9, 6], [7, 6, 8, 3, 9, 1, 4, 5, 0], [9, 2, 5, 6, 0, 0, 3, 7, 1], [5, 4, 3, 8, 6, 2, 9, 1, 7], [1, 9, 7, 4, 3, 5, 0, 2, 8], [2, 8, 6, 7, 1, 9, 5, 3, 4]] 

Solution of first sudoku:
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]


In [7]:
# Helper functions 
def get_possible_actions (sudoku):
    actions = []
    valid = True
    for i in range (0, len(sudoku)):#sudoku.shape[0]):
        for j in range (0, len(sudoku)):#sudoku.shape[0]):
            if sudoku[i][j] == 0:
                square_actions = get_possible_Square_values(sudoku, i,j)
                if len(square_actions) > 0:
                    actions.append( (i,j,square_actions,len(square_actions)) )
                    
                else:
                    # return empty array if spots no possible values for one square
                    return []
                
    ## order the square based on number of number of actions
    actions.sort(key=lambda tup: tup[3])
    ## returns the smallest list of possible actions
    if actions:
        return (actions[0])
    return []
    
def get_possible_Square_values(sudoku, i,j):
    ## get the possoble values of the square (i,j)
    # Predefined numbers
    domain = {1,2,3,4,5,6,7,8,9}
    
    # get row, col and unit values
    row  = sudoku[i]
    col  = [row[j] for row in sudoku ] 
    unit = get_unit_values(sudoku, [], i, j)
    
    
    # find the difference between complete values and the row, col, unit
    diff_row = domain.difference(row)
    diff_col = domain.difference(col)
    diff_unit = domain.difference(unit)
    
    # find the intersection of the results 
    possible_v = diff_col.intersection(diff_row)
    possible_v = possible_v.intersection(diff_unit)
    
    return list(possible_v)

def get_unit_values(sudoku, unit, i, j):
    
    encoding = ['a','a','a','b','b','b','c','c','c']
    
    ## gets the value in the unit    
    x = encoding.index(encoding[i]) 
    y = encoding.index(encoding[j])
    for square in range(0, 3):
        for i in range(0, 3):
            unit.append(sudoku[x][y])
            y+=1
        x+=1
        y = encoding.index(encoding[j])
    return unit

def isValid(sudoku):
    
    ## we want to check if a row and  is unique
    for i in range(0, len(sudoku)):#sudoku.shape[0]):
        
        ## uniqueness check here
        row = [(n) for n in sudoku[i] if n!=0]
        #unique, count = np.unique(row[row!=0], return_counts=True) 
        if len(row) > len(set(row)):#unique[count>1].size > 0:
            return False
        
        for j in range(0, len(sudoku)):#sudoku.shape[0]):
            col = [(r[j]) for r in sudoku if r[j]!=0]
            ## uniqueness check here
            #unique, count = np.unique(col[col!=0], return_counts=True) 
            if len(col) > len(set(col)):#unique[count>1].size > 0 :
                print(col)
                return False
    return True

def isInitialValid(sudoku): 
    
    ## we want to check if a row and  is unique
    for i in range(0, sudoku.shape[0]):
        
        ## uniqueness check here
        row = sudoku[i]
        unique, count = np.unique(row[row!=0], return_counts=True) 
        if len(row) > unique[count>1].size > 0:
            return False
        
        for j in range(0, sudoku.shape[0]):
            col = sudoku[:,j]
            ## uniqueness check here
            unique, count = np.unique(col[col!=0], return_counts=True) 
            if unique[count>1].size > 0 :
                return False
    return True
def get_next_sudoku(sudoku, i, j, action):
    sudoku[i][j] = action
    return sudoku

def is_in_arr(sudoku, arr):
    for item in arr:
        if np.array_equal(item, arr): 
            return True
    return False

In [5]:
# solver function 

import copy, itertools
def sudoku_solver(sudoku):
    
    if not isInitialValid(sudoku):
        return np.full((9,9),-1.)
        
    solved_sudoku = listSolver(sudoku.tolist())
    if not solved_sudoku:
        return np.full((9,9),-1.)
    return np.array(solved_sudoku)


def findNextCellToFill(grid):
    for x in range (0,9):
        for y in range(0,9):
            if grid[x][y] == 0:
                return x,y
    return -1,-1


def listSolver(grid,i=0,j=0):

    i, j = findNextCellToFill(grid)
    # return True when solved
    if i == -1:
        return True

    for action in get_possible_Square_values(grid, i, j):
            grid[i][j] = action

            if listSolver(grid ,i, j):
                return grid
#           backtrack
            grid[i][j]= 0
    return False 

#------------------------------------------ for debugging
# w = sudoku_solver(sudokus[5])
# print(w)




### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

In [6]:
# tests with varying difficulty 

SKIP_TESTS = False
if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
                
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.005029000000000061 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]


This is your solution for hard sudoku number 0
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.17871400000000004 seconds to solve.

This is hard sudoku number 1
[[1 0 0 7 0 0 0 0 0]
 [0 3 2 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0]
 [0 8 0 0 0 2 0 7 0]
 [5 0 7 0 0 1 0 0 0]
 [0 0 0 0 0 3 6 1 0]
 [7 0 0 0 0 0 2 0 9]
 [0 0 0 0 5 0 0 0 0]
 [3 0 0 0 0 4 0 0 5]]
This is your solution for hard sudoku number 1
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -

This is your solution for hard sudoku number 14
[[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1. -1. -1. -1. -1.]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 1.5419540000000005 seconds to solve.

15/15 hard sudokus correct
